In [1]:
%pip install numpy
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:

%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install pillow

Note: you may need to restart the kernel to use updated packages.


### ResNet

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

num_classes_2x2 = 4         # for 2x2 jigsaw puzzle there are 4 possible positions for each piece

# Load the pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False)

# Freeze the base model layers
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)

# Output layer for 4 classes
predictions_2x2 = Dense(4, activation='softmax', name='output_2x2')(x)

# Create a single model with both output layers
model = Model(inputs=base_model.input, outputs=predictions_2x2)

# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()




Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, None, None, 3)        0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, None, None, 64)       9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, None, None, 64)       256       ['conv1_conv[0][0]']          
 on)                                                                                       

In [5]:
# Dataset Source Credit: https://www.kaggle.com/datasets/shivajbd/jigsawpuzzle

# Load the Jigsaw Puzzle dataset and split it into train, test and valid sets
import os
import pandas as pd
import numpy as np
import PIL
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Define function to load dataset from csv files and images
def load_dataset(root_dir, csv_filename):
    data = pd.read_csv(csv_filename)
    images = []
    labels = []
    
    for _, row in data.iterrows():
        image_path = os.path.join(root_dir, row['image'])
        image = load_img(image_path, target_size=(100, 100))
        image = img_to_array(image) / 255.0
        images.append(image)

        # Convert string label to list of integers
        labels.append([int(x) for x in row['label'].split()])
    return images, labels

In [6]:
# Load data for 2x2 puzzle
train_images_2x2, train_labels_2x2 = load_dataset('../data/puzzle_2x2/train', '../data/puzzle_2x2/train.csv')
print("Train images 2x2 loaded: ", len(train_images_2x2))

Train images 2x2 loaded:  93136


In [7]:
test_images_2x2, test_labels_2x2 = load_dataset('../data/puzzle_2x2/test', '../data/puzzle_2x2/test.csv')
print("Test images 2x2 loaded: ", len(test_images_2x2))

Test images 2x2 loaded:  2176


In [8]:
valid_images_2x2, valid_labels_2x2 = load_dataset('../data/puzzle_2x2/valid', '../data/puzzle_2x2/valid.csv')
print("Valid images 2x2 loaded: ", len(valid_images_2x2))

Valid images 2x2 loaded:  385


In [9]:
# Convert lists to numpy arrays
train_data = np.array(train_images_2x2)
test_data = np.array(test_images_2x2)
valid_data = np.array(valid_images_2x2)

# Print shapes of the datasets
print("Train data shape:", train_data.shape)
print("Test data shape:", test_data.shape)
print("Validation data shape:", valid_data.shape)

Train data shape: (93136, 100, 100, 3)
Test data shape: (2176, 100, 100, 3)
Validation data shape: (385, 100, 100, 3)


In [10]:
# Convert lists to numpy arrays
train_label_arrays = [np.array(sublist) for sublist in train_labels_2x2]

# Find the maximum length of the nested lists
max_length = max(len(sublist) for sublist in train_labels_2x2)

# Pad the nested lists to ensure consistent lengths
padded_train_label_arrays = [sublist + [0] * (max_length - len(sublist)) for sublist in train_labels_2x2]

# Convert padded nested lists to a NumPy array
train_labels_array = np.array(padded_train_label_arrays)

print("Shape of train_labels_array:", train_labels_array.shape)
print("Size of flattened train_labels_array:", train_labels_array.size)

Shape of train_labels_array: (93136, 4)
Size of flattened train_labels_array: 372544


In [11]:
# Convert lists to numpy arrays
train_label_arrays = [np.array(sublist) for sublist in test_labels_2x2]

# Find the maximum length of the nested lists
max_length = max(len(sublist) for sublist in test_labels_2x2)

# Pad the nested lists to ensure consistent lengths
padded_test_label_arrays = [sublist + [0] * (max_length - len(sublist)) for sublist in test_labels_2x2]

# Convert padded nested lists to a NumPy array
test_labels_array = np.array(padded_test_label_arrays)

print("Shape of train_labels_array:", test_labels_array.shape)
print("Size of flattened train_labels_array:", test_labels_array.size)

Shape of train_labels_array: (2176, 4)
Size of flattened train_labels_array: 8704


In [12]:
# Convert lists to numpy arrays
train_label_arrays = [np.array(sublist) for sublist in valid_labels_2x2]

# Find the maximum length of the nested lists
max_length = max(len(sublist) for sublist in valid_labels_2x2)

# Pad the nested lists to ensure consistent lengths
padded_valid_label_arrays = [sublist + [0] * (max_length - len(sublist)) for sublist in valid_labels_2x2]

# Convert padded nested lists to a NumPy array
valid_labels_array = np.array(padded_valid_label_arrays)

print("Shape of train_labels_array:", valid_labels_array.shape)
print("Size of flattened train_labels_array:", valid_labels_array.size)

Shape of train_labels_array: (385, 4)
Size of flattened train_labels_array: 1540


In [13]:
# Normalize pixel values to [0, 1]
train_data = train_data / 255.0
test_data = test_data / 255.0
valid_data = valid_data / 255.0

# Train the model
history = model.fit(train_data, train_labels_array, epochs=10, validation_data=(valid_data, valid_labels_array))

# Evaluate the model
test_loss, test_acc = model.evaluate(test_data, test_labels_array)
print("Test accuracy:", test_acc)

# Make predictions
predictions = model.predict(test_data)

Epoch 1/10


2911/2911 [==============================] - 1125s 384ms/step - loss: 192.8692 - accuracy: 0.2511 - val_loss: 604.0864 - val_accuracy: 0.2312
Epoch 2/10
2911/2911 [==============================] - 1097s 377ms/step - loss: 506.1817 - accuracy: 0.2482 - val_loss: 530.7325 - val_accuracy: 0.2312
Epoch 3/10
2911/2911 [==============================] - 1104s 379ms/step - loss: 727.1923 - accuracy: 0.2465 - val_loss: 831.0805 - val_accuracy: 0.2312
Epoch 4/10
2911/2911 [==============================] - 1103s 379ms/step - loss: 911.7670 - accuracy: 0.2508 - val_loss: 523.2650 - val_accuracy: 0.2494
Epoch 5/10
2911/2911 [==============================] - 1094s 376ms/step - loss: 1181.5133 - accuracy: 0.2494 - val_loss: 945.6624 - val_accuracy: 0.2442
Epoch 6/10
2911/2911 [==============================] - 1100s 378ms/step - loss: 1392.7091 - accuracy: 0.2491 - val_loss: 2052.0000 - val_accuracy: 0.2442
Epoch 7/10
2911/2911 [==============================] - 1093s 375ms/step - lo